# Importing libraries and data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('googleplaystore.csv')
df.sample(5)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
2923,FilterGrid - Cam&Photo Editor,PHOTOGRAPHY,4.6,126337,9.6M,"1,000,000+",Free,0,Everyone,Photography,"March 15, 2017",2.0.5,4.0 and up
9926,Going Abroad,FAMILY,3.0,606,42M,"50,000+",Free,0,Everyone 10+,Educational,"August 16, 2016",1.2,4.0 and up
4935,Hi-Smart AC,TOOLS,2.2,32,9.4M,"1,000+",Free,0,Everyone,Tools,"April 3, 2018",1.1.0,4.4 and up
10510,FJ-link,TOOLS,NaN,1,20M,10+,Free,0,Everyone,Tools,"January 3, 2018",1.1.2,4.1 and up
1866,Lords Mobile: Battle of the Empires - Strategy...,GAME,4.4,3057481,14M,"50,000,000+",Free,0,Teen,Strategy,"July 26, 2018",1.76,2.3 and up


In [3]:
df['Size'].str[-1].unique()

array(['M', 'e', 'k', '+'], dtype=object)

# Data Cleaning

## 1.1 Investigation

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [5]:
df.duplicated().sum()

np.int64(483)

In [6]:
df.describe()

,Rating
count,9367.000000
mean,4.193338
std,0.537431
min,1.000000
25%,4.000000
50%,4.300000
75%,4.500000
max,19.000000


#### Notes
- 483 duplicated row
- `Price`, `Size`, and `Android Ver` columns are mixed with string 
    - `Size` column is in multiple units
    - `Size` and `Android Ver` columns have rows with the content "Varies with device"
- `Last Updated` column needs to be converted to date
- `Reviews` column needs to be converted to integer
- `Current Ver` co,umn won't be converted to float since it is refers to the development of each app indinvidually

## 1.2 Solving data issues

### Dropping duplicated Rows

In [7]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.duplicated().sum()

np.int64(0)

### Working with the Size column


In [8]:
df['Size'].value_counts()

Size
Varies with device    1526
11M                    188
13M                    186
12M                    186
14M                    182
                      ... 
552k                     1
885k                     1
1020k                    1
582k                     1
619k                     1
Name: count, Length: 462, dtype: int64

In [9]:
df[df['Size'].str.endswith('e')].sample(5)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
4312,Casino X - Free Online Slots,GAME,4.5,30515,Varies with device,"1,000,000+",Free,0,Teen,Casino,"May 22, 2018",2.92,4.0 and up
7925,Daily Horoscope,LIFESTYLE,4.7,407589,Varies with device,"10,000,000+",Free,0,Everyone,Lifestyle,"July 20, 2018",Varies with device,Varies with device
3901,MEGA,PRODUCTIVITY,4.0,549214,Varies with device,"50,000,000+",Free,0,Everyone,Productivity,"July 4, 2018",Varies with device,Varies with device
2365,bigbasket - online grocery,SHOPPING,4.3,216741,Varies with device,"5,000,000+",Free,0,Everyone,Shopping,"August 6, 2018",4.6.0,4.1 and up
2298,Whisper,SOCIAL,4.1,205803,Varies with device,"5,000,000+",Free,0,Teen,Social,"July 18, 2018",Varies with device,Varies with device


In [10]:
df[df['Size'].str.endswith('+')]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
9990,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


#### So i have noticed that this row is shifted to the right starting from the category 

In [11]:
# Fixing the shift in the row by keeping the first value which is the name then add null value to the category then the rest of the row
row = df.iloc[9990].copy()
new_row = [row[0]] + [np.nan] + list(row[1:-1])   # Takes from the second value to the one before last since the last is null
df.iloc[9990] = new_row

C:\Users\Youssif\AppData\Local\Temp\ipykernel_15180\903078688.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row = [row[0]] + [np.nan] + list(row[1:-1])   # Takes from the second value to the one before last since the last is null
C:\Users\Youssif\AppData\Local\Temp\ipykernel_15180\903078688.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.9' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[9990] = new_row


In [12]:
df.iloc[9990]

App               Life Made WI-Fi Touchscreen Photo Frame
Category                                              NaN
Rating                                                1.9
Reviews                                              19.0
Size                                                 3.0M
Installs                                           1,000+
Type                                                 Free
Price                                                   0
Content Rating                                   Everyone
Genres                                                NaN
Last Updated                            February 11, 2018
Current Ver                                        1.0.19
Android Ver                                    4.0 and up
Name: 9990, dtype: object

- now all the values ends with 'e' which is varies with device will be replaced with NaN
- the size column will be converted to float
- The whole column will be in 'M' unit
    - M = K / 1024

In [13]:
def cleanSize(size):
    if size.endswith('e'):
        size = np.nan
        return size
    if size.endswith('k'):
        size = float(size.strip('k')) / 1024
        return size
    size = float(size.strip('M'))
    return size

df['Size'] = df['Size'].apply(cleanSize)
df['Size'].sample(20)

363           NaN
5656     2.600000
9438    34.000000
4880     0.259766
8053    11.000000
2573    27.000000
1060     7.100000
7546     0.262695
7058     0.487305
8935     0.765625
4127     0.306641
3492    91.000000
3778    23.000000
1389    39.000000
5595    53.000000
2567    50.000000
4940          NaN
3991    22.000000
8653     3.800000
5885    30.000000
Name: Size, dtype: float64

In [14]:
df["Size"].isna().sum()

np.int64(1526)

- Now all the values of the Size column are float and in 'M' unit
- There is 1527 null values in the Size column

### Converting the rest of numerical columns to float

In [15]:
df['Price'].value_counts()
# this column is mostly zeros (Exactly 88%) but i will keep it cause it may give insights on other columns like intalls or rating


Price
0          9593
$0.99       146
$2.99       125
$1.99        73
$4.99        70
           ... 
$3.61         1
$394.99       1
$1.26         1
$1.20         1
$1.04         1
Name: count, Length: 92, dtype: int64

In [16]:
df["Price"].unique()

array(['0', '$4.99', '$3.99', '$6.99', '$1.49', '$2.99', '$7.99', '$5.99',
       '$3.49', '$1.99', '$9.99', '$7.49', '$0.99', '$9.00', '$5.49',
       '$10.00', '$24.99', '$11.99', '$79.99', '$16.99', '$14.99',
       '$1.00', '$29.99', '$12.99', '$2.49', '$10.99', '$1.50', '$19.99',
       '$15.99', '$33.99', '$74.99', '$39.99', '$3.95', '$4.49', '$1.70',
       '$8.99', '$2.00', '$3.88', '$25.99', '$399.99', '$17.99',
       '$400.00', '$3.02', '$1.76', '$4.84', '$4.77', '$1.61', '$2.50',
       '$1.59', '$6.49', '$1.29', '$5.00', '$13.99', '$299.99', '$379.99',
       '$37.99', '$18.99', '$389.99', '$19.90', '$8.49', '$1.75',
       '$14.00', '$4.85', '$46.99', '$109.99', '$154.99', '$3.08',
       '$2.59', '$4.80', '$1.96', '$19.40', '$3.90', '$4.59', '$15.46',
       '$3.04', '$4.29', '$2.60', '$3.28', '$4.60', '$28.99', '$2.95',
       '$2.90', '$1.97', '$200.00', '$89.99', '$2.56', '$30.99', '$3.61',
       '$394.99', '$1.26', '$1.20', '$1.04'], dtype=object)

> the `Price` column is mostly zeros (Exactly 88%) but i will keep it cause it may give insights on other columns like installs or rating

In [ ]:
# Converting Price to float
df["Price"] = df["Price"].apply(lambda x : float(x.strip('$')))
df["Price"].unique()

array([  0.  ,   4.99,   3.99,   6.99,   1.49,   2.99,   7.99,   5.99,
         3.49,   1.99,   9.99,   7.49,   0.99,   9.  ,   5.49,  10.  ,
        24.99,  11.99,  79.99,  16.99,  14.99,   1.  ,  29.99,  12.99,
         2.49,  10.99,   1.5 ,  19.99,  15.99,  33.99,  74.99,  39.99,
         3.95,   4.49,   1.7 ,   8.99,   2.  ,   3.88,  25.99, 399.99,
        17.99, 400.  ,   3.02,   1.76,   4.84,   4.77,   1.61,   2.5 ,
         1.59,   6.49,   1.29,   5.  ,  13.99, 299.99, 379.99,  37.99,
        18.99, 389.99,  19.9 ,   8.49,   1.75,  14.  ,   4.85,  46.99,
       109.99, 154.99,   3.08,   2.59,   4.8 ,   1.96,  19.4 ,   3.9 ,
         4.59,  15.46,   3.04,   4.29,   2.6 ,   3.28,   4.6 ,  28.99,
         2.95,   2.9 ,   1.97, 200.  ,  89.99,   2.56,  30.99,   3.61,
       394.99,   1.26,   1.2 ,   1.04])

In [19]:
# Converting Reviews column to float
df["Reviews"] = df['Reviews']. apply(lambda x : int(x))
df["Reviews"]

0           159
1           967
2         87510
3        215644
4           967
          ...  
10353        38
10354         4
10355         3
10356       114
10357    398307
Name: Reviews, Length: 10358, dtype: int64

In [20]:
df["Android Ver"].unique()

array(['4.0.3 and up', '4.2 and up', '4.4 and up', '2.3 and up',
       '3.0 and up', '4.1 and up', '4.0 and up', '2.3.3 and up',
       'Varies with device', '2.2 and up', '5.0 and up', '6.0 and up',
       '1.6 and up', '1.5 and up', '2.1 and up', '7.0 and up',
       '5.1 and up', '4.3 and up', '4.0.3 - 7.1.1', '2.0 and up',
       '3.2 and up', '4.4W and up', '7.1 and up', '7.0 - 7.1.1',
       '8.0 and up', '5.0 - 8.0', '3.1 and up', '2.0.1 and up',
       '4.1 - 7.1.1', nan, '5.0 - 6.0', '1.0 and up', '2.2 - 7.1.1',
       '5.0 - 7.1.1'], dtype=object)

In [ ]:
def clean_android_ver(version):
    pass